# `speciesgrids` product

[speciesgrids](https://github.com/iobis/speciesgrids) is a Python package to build WoRMS aligned combined OBIS and GBIF species distribution datasets. The resulting dataset is available in a few resolutions on AWS S3. The dataset can be downloaded locally for best performance, or queried directly from the S3 bucket. For more details about downloading and using the dataset, see the [speciesgrids README](https://github.com/iobis/speciesgrids).

The product aggregate all marine data from the two major databases, in a very fine grid using Uber's H3 system. H3 is an hierarchical grid system, and that enables both very fast queries and aggregations. To learn more about Uber's H3 system check [here.](https://h3geo.org/)

<img src="https://h3geo.org/images/parent-child.png" height=200></img>

The dataset is served as a `parquet` file. [Parquet](https://parquet.apache.org/) is a column-based performant and light format developed by Apache. We can work with `parquet` files on R and Python using the `arrow` [package.](https://arrow.apache.org/docs/r/index.html). You can learn more about parquet in [this tutorial](https://resources.obis.org/tutorials/arrow-obis/).